In [1]:
pip install numpy
pip install opencv-python
pip install torch
pip install torchvision
pip install cython
pip install lap
pip install filterpy
pip install scipy
pip install deep_sort_realtime


SyntaxError: invalid syntax (3999969525.py, line 1)

In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install torch

In [2]:
pip install torchvision


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [3]:
pip install cython

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lap

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install filterpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/7.8 MB 11.7 MB/s eta 0:00:01
   ------------------------ --------------- 4.7/7.8 MB 12.4 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 13.0 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 13.0 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 13.0 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 13.0 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 11.3 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.w

In [6]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install deep_sort_realtime

   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.4 MB 4.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/8.4 MB 7.3 MB/s eta 0:00:01
   ---------------------- ----------------- 4.7/8.4 MB 9.2 MB/s eta 0:00:01
   --------------------------------- ------ 7.1/8.4 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 8.4/8.4 MB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### второй вариант обвода лиц в квадрат ####
очень точно определяет лицо и заодно всего человека. не сбоит при повороте лица, очках на лице и закрытии большей части лица.

In [9]:
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

# Загрузка YOLOv5 модели
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Инициализация DeepSort трекера
tracker = DeepSort(max_age=30, n_init=2, nn_budget=100)

# Функция для детекции лиц
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Функция для обработки видео
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Обнаружение объектов с использованием YOLOv5
        results = model(frame)

        # Извлечение объектов и их координат
        detections = []
        for *box, conf, cls in results.xyxy[0].cpu().numpy():
            if cls == 0:  # Класс 0 - это человек
                x1, y1, x2, y2 = map(int, box)
                detections.append(([x1, y1, x2-x1, y2-y1], conf))

        # Отслеживание объектов с использованием DeepSort
        tracks = tracker.update_tracks(detections, frame=frame)

        # Обводим всех людей в кадре
        for track in tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
            cv2.putText(frame, f'ID: {track.track_id}', (int(bbox[0]), int(bbox[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Обнаружение лиц и обведение их в квадраты
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Отображение видеопотока
        cv2.imshow('Video', frame)

        # Выход по нажатию клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Запуск функции обработки видео
process_video(0)


Using cache found in C:\Users\ester/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-12 Python-3.12.2 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\ester/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ester/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ester/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ester/.cache\torch\hub\ultralytics_yolov5_m